In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
from pandas import get_dummies
from sklearn.preprocessing import OneHotEncoder
import spacy
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score

In [4]:
trueDf=pd.read_csv("/content/drive/MyDrive/Data_AI/Fake and real news/True.csv",nrows=5000)
fakeDf=pd.read_csv("/content/drive/MyDrive/Data_AI/Fake and real news/Fake.csv",nrows=5000)
trueDf.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
trueDf['subject'].unique()

array(['politicsNews'], dtype=object)

In [6]:
trueDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    5000 non-null   object
 1   text     5000 non-null   object
 2   subject  5000 non-null   object
 3   date     5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [7]:
trueDf.describe()

,title,text,subject,date
count,5000,5000,5000,5000
unique,4955,4962,1,294
top,Highlights: The Trump presidency on April 13 a...,(Reuters) - Highlights for U.S. President Dona...,politicsNews,"April 7, 2017"
freq,8,8,5000,73


In [8]:
fakeDf.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
fakeDf['subject'].unique()

array(['News'], dtype=object)

In [10]:
trueDf['T/F']=1
fakeDf['T/F']=0
X=pd.concat([trueDf,fakeDf],axis=0)
y=pd.get_dummies(X['subject'],dtype='int8')
X.drop(columns=['date','subject'],axis=1,inplace=True)
X=pd.concat([X,y],axis=1)
y=X['T/F']
X.drop(columns=['T/F'],axis=1,inplace=True)
X.shape

(10000, 4)

In [11]:
def preprocessingTextUsingSPACY(text):
    text=text.lower()
    lemWithStop=spacy.load("en_core_web_sm")
    doc=lemWithStop(text)
    #removal of stopwords and Lemmatizing
    if doc!=None and len(doc):
        text= " ".join([ word.lemma_ for word in doc if not word.is_stop ])
    else:
        print(text)
    return text

In [12]:
cv=CountVectorizer()

In [13]:
title=[(' '.join([str(el).replace(" ","") for el in list(ngrams(ti.split(),2))])) for ti in X['title']]
titleArr=cv.fit_transform(title).toarray()
titleDF=pd.DataFrame(titleArr,dtype='int8')

text=[' '.join([str(el).replace(" ","") for el in list(ngrams(ti.split(),2))]) for ti in X['text']]
X.drop(columns=['title','text'],inplace=True)
textArr=cv.fit_transform(text).toarray()
textDF=pd.DataFrame(textArr,dtype='int16')

titleDF.reset_index(inplace=True, drop=True)
textDF.reset_index(inplace=True, drop=True)
X.reset_index(inplace=True, drop=True)
temp=pd.concat([X,titleDF,textDF],axis=1)
temp.head()

,News,politicsNews,0,1,2,3,4,5,6,7,...,58312,58313,58314,58315,58316,58317,58318,58319,58320,58321
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
temp.describe()

,News,politicsNews,0,1,2,3,4,5,6,7,...,58312,58313,58314,58315,58316,58317,58318,58319,58320,58321
count,10000.000000,10000.000000,10000.000000,10000.0000,10000.000000,10000.000000,10000.000000,10000.0000,10000.0000,10000.0000,...,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.0000,10000.000000,10000.0000,10000.0000
mean,0.500000,0.500000,0.004100,0.0002,0.000400,0.000600,0.001600,0.0002,0.0002,0.0002,...,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.0002,0.000600,0.0002,0.0002
std,0.500025,0.500025,0.094255,0.0200,0.028283,0.034638,0.056549,0.0200,0.0200,0.0200,...,0.0200,0.0200,0.0200,0.0200,0.0200,0.0200,0.0200,0.034638,0.0200,0.0200
min,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0000
25%,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0000
50%,0.500000,0.500000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0000
75%,1.000000,1.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,0.0000
max,1.000000,1.000000,4.000000,2.0000,2.000000,2.000000,2.000000,2.0000,2.0000,2.0000,...,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.000000,2.0000,2.0000


In [15]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 68421 entries, News to 58321
dtypes: int16(58322), int8(10099)
memory usage: 1.2 GB


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(temp.values,y,random_state=20,test_size=0.2)
gnb=GaussianNB()
gnb.fit(X_train,y_train)

In [ ]:
y_pred=gnb.predict(X_test)
confusion_matrix(y_pred=y_pred,y_true=y_test)

In [ ]:
accuracy_score(y_pred=y_pred,y_true=y_test)